In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [4]:
%load_ext autoreload
%autoreload 2
from data.api_fetcher import ApiFetcher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
api = ApiFetcher(starting_year=2019, ending_year=2025)
df = api.get_dataframe('leaguegamelog')

In [6]:
print(df.head)

<bound method NDFrame.head of          game_id  home_fga  away_fga  home_fg_pct  away_fg_pct  home_fg3a  \
0     0021900001       103       102        0.408        0.422         40   
1     0021900002        81        85        0.519        0.435         31   
2     0021900005        86        88        0.430        0.375         30   
3     0021900009        76        93        0.461        0.398         36   
4     0021900003        88       105        0.511        0.467         44   
...          ...       ...       ...          ...          ...        ...   
7049  0022401188        74        86        0.514        0.430         35   
7050  0022401187        90        85        0.389        0.376         45   
7051  0022401195        98        89        0.469        0.449         43   
7052  0022401194       104        87        0.500        0.437         40   
7053  0022401197        84        97        0.512        0.474         32   

      away_fg3a  home_fg3_pct  away_fg3_pct  

In [10]:
# ----------------------------------------------------------
# Split into train / val / test (chronological)
# ----------------------------------------------------------
n = len(df)
train_df = df.iloc[:int(n*0.7)]
val_df   = df.iloc[int(n*0.7):int(n*0.85)]
test_df  = df.iloc[int(n*0.85):]

# Map team IDs to contiguous indices
all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
n_teams = len(team2idx)

def df_to_arrays(df):
    home = df['home_team_id'].map(team2idx).values
    away = df['away_team_id'].map(team2idx).values
    y_home = df['home_pts'].values.astype(np.float32)
    y_away = df['away_pts'].values.astype(np.float32)
    return home, away, y_home, y_away

train_home, train_away, train_y_home, train_y_away = df_to_arrays(train_df)
val_home, val_away, val_y_home, val_y_away         = df_to_arrays(val_df)
test_home, test_away, test_y_home, test_y_away     = df_to_arrays(test_df)

# Dataset and loaders
train_ds = GamesDataset(train_home, train_away, train_y_home, train_y_away)
val_ds   = GamesDataset(val_home, val_away, val_y_home, val_y_away)
test_ds  = GamesDataset(test_home, test_away, test_y_home, test_y_away)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=False)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

# ----------------------------------------------------------
# Training loop (unchanged)
# ----------------------------------------------------------
for epoch in range(n_epochs):
    model.train()
    train_losses = []
    for home, away, y in train_loader:
        home, away, y = home.to(device), away.to(device), y.to(device)
        preds = model(home, away)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    print(f"Epoch {epoch+1}: train MSE={np.mean(train_losses):.4f}")

    # ---- Validation ----
    model.eval()
    with torch.no_grad():
        val_preds, val_targets = [], []
        for home, away, y in val_loader:
            home, away = home.to(device), away.to(device)
            preds = model(home, away)
            val_preds.append(preds.cpu())
            val_targets.append(y)
        val_preds = torch.cat(val_preds)
        val_targets = torch.cat(val_targets)
        mse_home = mean_squared_error(val_targets[:,0], val_preds[:,0])
        mse_away = mean_squared_error(val_targets[:,1], val_preds[:,1])
        print(f"          val MSE home={mse_home:.2f}, away={mse_away:.2f}")

# ----------------------------------------------------------
# Test set evaluation
# ----------------------------------------------------------
model.eval()
with torch.no_grad():
    test_preds, test_targets = [], []
    for home, away, y in test_loader:
        home, away = home.to(device), away.to(device)
        preds = model(home, away)
        test_preds.append(preds.cpu())
        test_targets.append(y)
    test_preds = torch.cat(test_preds)
    test_targets = torch.cat(test_targets)
    mse_home_test = mean_squared_error(test_targets[:,0], test_preds[:,0])
    mse_away_test = mean_squared_error(test_targets[:,1], test_preds[:,1])
    print(f"Test MSE home={mse_home_test:.2f}, away={mse_away_test:.2f}")


Epoch 1: train MSE=155.5180
          val MSE home=162.51, away=165.82
Epoch 2: train MSE=155.0041
          val MSE home=162.04, away=166.13
Epoch 3: train MSE=154.3348
          val MSE home=161.11, away=165.73
Epoch 4: train MSE=153.5758
          val MSE home=160.40, away=165.63
Epoch 5: train MSE=152.9573
          val MSE home=159.77, away=165.45
Epoch 6: train MSE=152.2105
          val MSE home=159.34, away=165.28
Epoch 7: train MSE=151.4144
          val MSE home=159.19, away=165.01
Epoch 8: train MSE=150.8591
          val MSE home=159.04, away=164.87
Epoch 9: train MSE=150.4451
          val MSE home=158.85, away=164.79
Epoch 10: train MSE=150.0322
          val MSE home=158.77, away=164.83
Epoch 11: train MSE=149.6352
          val MSE home=158.75, away=164.81
Epoch 12: train MSE=149.3102
          val MSE home=158.77, away=164.85
Epoch 13: train MSE=149.0213
          val MSE home=158.82, away=164.86
Epoch 14: train MSE=148.7670
          val MSE home=158.87, away=164.84
E

In [12]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

class TeamEmbeddingModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=32, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(0.0))  # Learnable home advantage
        
        # Network to process team embeddings
        self.net = nn.Sequential(
            nn.Linear(embedding_dim * 2 + 1, hidden_dim),  # +1 for home advantage
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 2)  # home_score, away_score
        )
        
    def forward(self, home_team_ids, away_team_ids):
        home_emb = self.team_embeddings(home_team_ids)
        away_emb = self.team_embeddings(away_team_ids)
        
        # Add home advantage as a feature
        batch_size = home_team_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        
        # Concatenate embeddings + home advantage
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        scores = self.net(x)
        return scores

class OnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TeamEmbeddingModel(n_teams).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.loss_fn = nn.MSELoss()
        
        # Track performance metrics
        self.predictions = []
        self.actuals = []
        self.game_dates = []
        
    def predict_game(self, home_team_id, away_team_id):
        """
        Make prediction using only team IDs (no game stats)
        Returns: (home_score_pred, away_score_pred)
        """
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_team_id], device=self.device)
            away_tensor = torch.tensor([away_team_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0, 0].item(), pred[0, 1].item()
    
    def update_with_result(self, home_team_id, away_team_id, actual_home_score, actual_away_score, 
                          game_stats=None, learning_rate=None):
        """
        Update model weights after game completion
        game_stats: dict of additional game statistics for richer learning (optional)
        """
        if learning_rate:
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = learning_rate
                
        self.model.train()
        
        # Prepare tensors
        home_tensor = torch.tensor([home_team_id], device=self.device)
        away_tensor = torch.tensor([away_team_id], device=self.device)
        target = torch.tensor([[actual_home_score, actual_away_score]], 
                             dtype=torch.float32, device=self.device)
        
        # Forward pass
        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target)
        
        # Backward pass
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def evaluate_performance(self):
        """Calculate current prediction accuracy"""
        if not self.predictions:
            return None
            
        home_preds = [p[0] for p in self.predictions]
        away_preds = [p[1] for p in self.predictions]
        home_actuals = [a[0] for a in self.actuals]
        away_actuals = [a[1] for a in self.actuals]
        
        home_mse = mean_squared_error(home_actuals, home_preds)
        away_mse = mean_squared_error(away_actuals, away_preds)
        
        return {
            'home_mse': home_mse,
            'away_mse': away_mse,
            'total_games': len(self.predictions)
        }

def chronological_online_training(df, team2idx, initial_train_games=500, device='cpu'):
    """
    Train model chronologically with online learning
    """
    # Sort by date to ensure chronological order
    df_sorted = df.sort_values('date').reset_index(drop=True)
    
    n_teams = len(team2idx)
    predictor = OnlineNBAPredictor(n_teams, device)
    
    # Initial batch training on first N games
    print(f"Initial training on first {initial_train_games} games...")
    initial_df = df_sorted.iloc[:initial_train_games]
    
    # Create initial training data (just team IDs and scores)
    initial_home = torch.tensor([team2idx[tid] for tid in initial_df['home_team_id']], device=device)
    initial_away = torch.tensor([team2idx[tid] for tid in initial_df['away_team_id']], device=device)
    initial_targets = torch.tensor(initial_df[['home_pts', 'away_pts']].values, 
                                 dtype=torch.float32, device=device)
    
    # Initial training loop
    predictor.model.train()
    for epoch in range(100):  # Quick initial training
        preds = predictor.model(initial_home, initial_away)
        loss = predictor.loss_fn(preds, initial_targets)
        predictor.optimizer.zero_grad()
        loss.backward()
        predictor.optimizer.step()
        
        if epoch % 20 == 0:
            print(f"  Epoch {epoch}: Loss = {loss.item():.4f}")
    
    # Online learning on remaining games
    print(f"\nStarting online learning on remaining {len(df_sorted) - initial_train_games} games...")
    
    performance_history = []
    
    for idx in range(initial_train_games, len(df_sorted)):
        game = df_sorted.iloc[idx]
        
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']
        
        # 1. Make prediction (before knowing result)
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        
        # Store prediction and actual for evaluation
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))
        predictor.game_dates.append(game['date'])
        
        # 2. Update model with actual result
        # Adaptive learning rate - reduce over time
        adaptive_lr = max(0.0001, 0.001 * (0.999 ** (idx - initial_train_games)))
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away, 
                                          learning_rate=adaptive_lr)
        
        # Periodic evaluation
        if (idx - initial_train_games) % 100 == 0:
            perf = predictor.evaluate_performance()
            performance_history.append(perf)
            print(f"Game {idx}: Home MSE={perf['home_mse']:.2f}, Away MSE={perf['away_mse']:.2f}, "
                  f"LR={adaptive_lr:.6f}")
    
    return predictor, performance_history

# Usage example:
def run_online_nba_prediction(df):
    """
    Main function to run the online learning system
    """
    # Create team mapping from full dataset (this is OK since we're not using it for features)
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    idx2team = {i: tid for tid, i in team2idx.items()}
    
    # Run online training
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor, history = chronological_online_training(df, team2idx, device=device)
    
    # Final evaluation
    final_perf = predictor.evaluate_performance()
    print(f"\nFinal Performance:")
    print(f"Home MSE: {final_perf['home_mse']:.2f}")
    print(f"Away MSE: {final_perf['away_mse']:.2f}")
    print(f"Total games predicted: {final_perf['total_games']}")
    
    return predictor, team2idx, history

# Example of how to make a new prediction:
def predict_new_game(predictor, team2idx, home_team_name, away_team_name):
    """
    Predict a new game given team names
    """
    # You'd need a mapping from team names to IDs
    # For now, using team IDs directly
    home_id = team2idx[home_team_name]  # Assuming team_name is actually team_id
    away_id = team2idx[away_team_name]
    
    pred_home, pred_away = predictor.predict_game(home_id, away_id)
    print(f"Prediction: Home {home_team_name}: {pred_home:.1f}, Away {away_team_name}: {pred_away:.1f}")
    
    return pred_home, pred_away

# After the game is played, update the model:
def update_after_game(predictor, team2idx, home_team_name, away_team_name, 
                     actual_home_score, actual_away_score):
    """
    Update model after game result is known
    """
    home_id = team2idx[home_team_name]
    away_id = team2idx[away_team_name]
    
    loss = predictor.update_with_result(home_id, away_id, actual_home_score, actual_away_score)
    print(f"Model updated. Training loss: {loss:.4f}")
    
    return loss

# Run the system:
predictor, team2idx, history = run_online_nba_prediction(df)

Initial training on first 500 games...
  Epoch 0: Loss = 12328.2188
  Epoch 20: Loss = 12027.4609
  Epoch 40: Loss = 10894.0469
  Epoch 60: Loss = 7429.8105
  Epoch 80: Loss = 2352.1619

Starting online learning on remaining 6554 games...
Game 500: Home MSE=2.49, Away MSE=591.87, LR=0.001000
Game 600: Home MSE=623.94, Away MSE=630.08, LR=0.000905
Game 700: Home MSE=619.51, Away MSE=591.69, LR=0.000819
Game 800: Home MSE=504.13, Away MSE=487.14, LR=0.000741
Game 900: Home MSE=461.84, Away MSE=430.76, LR=0.000670
Game 1000: Home MSE=404.55, Away MSE=377.83, LR=0.000606
Game 1100: Home MSE=372.27, Away MSE=344.21, LR=0.000549
Game 1200: Home MSE=353.54, Away MSE=327.41, LR=0.000496
Game 1300: Home MSE=335.22, Away MSE=312.87, LR=0.000449
Game 1400: Home MSE=316.70, Away MSE=301.40, LR=0.000406
Game 1500: Home MSE=302.29, Away MSE=288.98, LR=0.000368
Game 1600: Home MSE=289.40, Away MSE=276.82, LR=0.000333
Game 1700: Home MSE=278.62, Away MSE=269.08, LR=0.000301
Game 1800: Home MSE=270.42,

In [15]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# Model: True online NBA predictor
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=32, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))  # Initial prior
        
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)
        
        self.net = nn.Sequential(
            nn.Linear(embedding_dim * 2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )
        
    def forward(self, home_team_ids, away_team_ids):
        home_emb = self.team_embeddings(home_team_ids)
        away_emb = self.team_embeddings(away_team_ids)
        batch_size = home_team_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA range
        return scores

# ==========================================================
# Online predictor using TrueOnlineNBAModel
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_team_id, away_team_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_team_id], device=self.device)
            away_tensor = torch.tensor([away_team_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0, 0].item(), pred[0, 1].item()

    def update_with_result(self, home_team_id, away_team_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_team_id], device=self.device)
        away_tensor = torch.tensor([away_team_id], device=self.device)
        target = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()

        return loss.item()

# ==========================================================
# Example: run online predictions
# ==========================================================
def run_online_model(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    n_teams = len(team2idx)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    predictor = TrueOnlineNBAPredictor(n_teams, device)
    total_loss = 0

    for idx, game in df_sorted.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        # Predict
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        # Update
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        # Print every 200 games
        if (idx + 1) % 200 == 0:
            recent_preds = predictor.predictions[-200:]
            recent_actuals = predictor.actuals[-200:]
            home_preds = [p[0] for p in recent_preds]
            home_actuals = [a[0] for a in recent_actuals]
            mse = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
            print(f"Games {idx+1}: Recent MSE={mse:.2f}, Avg Loss={total_loss/(idx+1):.4f}")

# Usage:
run_online_model(df)


Games 200: Recent MSE=185.47, Avg Loss=184.9017
Games 400: Recent MSE=184.79, Avg Loss=179.0451
Games 600: Recent MSE=117.27, Avg Loss=161.9040
Games 800: Recent MSE=176.30, Avg Loss=161.6079
Games 1000: Recent MSE=164.44, Avg Loss=159.4268
Games 1200: Recent MSE=168.29, Avg Loss=159.8294
Games 1400: Recent MSE=147.57, Avg Loss=159.0075
Games 1600: Recent MSE=151.77, Avg Loss=157.2459
Games 1800: Recent MSE=158.32, Avg Loss=157.4189
Games 2000: Recent MSE=140.70, Avg Loss=157.1467
Games 2200: Recent MSE=187.02, Avg Loss=159.4503
Games 2400: Recent MSE=143.10, Avg Loss=157.1764
Games 2600: Recent MSE=145.77, Avg Loss=156.7331
Games 2800: Recent MSE=150.04, Avg Loss=157.0810
Games 3000: Recent MSE=139.55, Avg Loss=157.0011
Games 3200: Recent MSE=149.78, Avg Loss=156.8298
Games 3400: Recent MSE=153.92, Avg Loss=156.6700
Games 3600: Recent MSE=134.62, Avg Loss=155.4280
Games 3800: Recent MSE=139.14, Avg Loss=154.4803
Games 4000: Recent MSE=148.56, Avg Loss=153.1312
Games 4200: Recent MSE=1

In [17]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. Model: Online NBA predictor with GRU-like embeddings
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=32, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim * 2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, home_team_ids, away_team_ids):
        home_emb = self.team_embeddings(home_team_ids)
        away_emb = self.team_embeddings(away_team_ids)
        batch_size = home_team_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA scoring range
        return scores

# ==========================================================
# 2. Online predictor
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_team_id, away_team_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_team_id], device=self.device)
            away_tensor = torch.tensor([away_team_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0, 0].item(), pred[0, 1].item()

    def update_with_result(self, home_team_id, away_team_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_team_id], device=self.device)
        away_tensor = torch.tensor([away_team_id], device=self.device)
        target = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()
        return loss.item()

# ==========================================================
# 3. Online training with chronological train/val/test
# ==========================================================
def run_online_train_val_test(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Split chronologically to avoid data leakage
    train_idx = int(n_total * 0.7)
    val_idx = int(n_total * 0.85)

    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # Map team IDs
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    n_teams = len(team2idx)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = TrueOnlineNBAPredictor(n_teams, device)

    # ----------- TRAIN (online, chronological) -----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % 200 == 0:
            recent_preds = predictor.predictions[-200:]
            recent_actuals = predictor.actuals[-200:]
            home_preds = [p[0] for p in recent_preds]
            home_actuals = [a[0] for a in recent_actuals]
            mse = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
            print(f"Train Games {idx+1}: Recent MSE={mse:.2f}, Avg Loss={total_loss/(idx+1):.4f}")

    # ----------- VALIDATION (chronological, no updates) -----------
    val_preds, val_actuals = [], []
    for idx, game in df_val.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append((pred_home, pred_away))
        val_actuals.append((actual_home, actual_away))

    home_preds = [p[0] for p in val_preds]
    home_actuals = [a[0] for a in val_actuals]
    mse_home = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
    away_preds = [p[1] for p in val_preds]
    away_actuals = [a[1] for a in val_actuals]
    mse_away = np.mean([(p-a)**2 for p, a in zip(away_preds, away_actuals)])
    print(f"Validation MSE: home={mse_home:.2f}, away={mse_away:.2f}")

    # ----------- TEST (chronological, final evaluation) -----------
    test_preds, test_actuals = [], []
    for idx, game in df_test.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append((pred_home, pred_away))
        test_actuals.append((actual_home, actual_away))

    home_preds = [p[0] for p in test_preds]
    home_actuals = [a[0] for a in test_actuals]
    mse_home = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
    away_preds = [p[1] for p in test_preds]
    away_actuals = [a[1] for a in test_actuals]
    mse_away = np.mean([(p-a)**2 for p, a in zip(away_preds, away_actuals)])
    print(f"Test MSE: home={mse_home:.2f}, away={mse_away:.2f}")

# Usage:
run_online_train_val_test(df)


Train Games 200: Recent MSE=197.00, Avg Loss=189.5830
Train Games 400: Recent MSE=188.11, Avg Loss=184.6207
Train Games 600: Recent MSE=117.52, Avg Loss=166.0097
Train Games 800: Recent MSE=178.35, Avg Loss=164.8056
Train Games 1000: Recent MSE=164.16, Avg Loss=161.9796
Train Games 1200: Recent MSE=168.33, Avg Loss=161.9608
Train Games 1400: Recent MSE=147.57, Avg Loss=160.8350
Train Games 1600: Recent MSE=150.76, Avg Loss=158.8364
Train Games 1800: Recent MSE=158.64, Avg Loss=158.8536
Train Games 2000: Recent MSE=140.60, Avg Loss=158.4280
Train Games 2200: Recent MSE=186.76, Avg Loss=160.5801
Train Games 2400: Recent MSE=143.07, Avg Loss=158.2138
Train Games 2600: Recent MSE=145.77, Avg Loss=157.6902
Train Games 2800: Recent MSE=149.74, Avg Loss=157.9536
Train Games 3000: Recent MSE=140.85, Avg Loss=157.8670
Train Games 3200: Recent MSE=149.73, Avg Loss=157.6445
Train Games 3400: Recent MSE=153.92, Avg Loss=157.4366
Train Games 3600: Recent MSE=134.45, Avg Loss=156.0715
Train Games 38

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from collections import defaultdict

# ==========================================================
# 1. Model: GRU-based NBA predictor
# ==========================================================
class GRUOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=32, hidden_dim=64, gru_layers=2):
        super().__init__()
        self.n_teams = n_teams
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_layers = gru_layers
        
        # Team embeddings
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)
        
        # GRU for processing team history sequences
        # Input: [team_embedding + opponent_embedding + home_flag + prev_score_diff]
        self.gru_input_dim = embedding_dim * 2 + 2  # team + opponent + home_flag + score_diff
        self.team_gru = nn.GRU(
            input_size=self.gru_input_dim,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0.1 if gru_layers > 1 else 0
        )
        
        # Final prediction network
        self.prediction_net = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim),  # home_hidden + away_hidden + home_advantage
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)  # home_score, away_score
        )
        
    def get_team_representation(self, team_history, team_id):
        """Get GRU representation for a team based on their recent games"""
        if len(team_history) == 0:
            # If no history, return zero hidden state
            return torch.zeros(1, self.hidden_dim, device=next(self.parameters()).device)
        
        # Convert history to tensor
        history_tensor = torch.stack(team_history).unsqueeze(0)  # [1, seq_len, features]
        
        # Pass through GRU
        gru_output, hidden = self.team_gru(history_tensor)
        
        # Return the final hidden state (last timestep)
        return hidden[-1]  # [1, hidden_dim] - take last layer
        
    def forward(self, home_team_id, away_team_id, home_history, away_history):
        """
        Args:
            home_team_id: int
            away_team_id: int  
            home_history: list of tensors representing recent games for home team
            away_history: list of tensors representing recent games for away team
        """
        device = next(self.parameters()).device
        
        # Get team representations from GRU
        home_repr = self.get_team_representation(home_history, home_team_id)  # [1, hidden_dim]
        away_repr = self.get_team_representation(away_history, away_team_id)  # [1, hidden_dim]
        
        # Add home advantage
        home_adv = self.home_advantage.unsqueeze(0).unsqueeze(0)  # [1, 1]
        
        # Concatenate features
        x = torch.cat([home_repr, away_repr, home_adv], dim=1)  # [1, hidden_dim*2 + 1]
        
        # Predict scores
        raw_scores = self.prediction_net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA scoring range
        
        return scores

# ==========================================================
# 2. GRU Online predictor
# ==========================================================
class GRUOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu', history_length=10):
        self.device = device
        self.history_length = history_length
        self.model = GRUOnlineNBAModel(n_teams).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        
        # Track game history for each team (chronologically)
        self.team_histories = defaultdict(list)  # team_id -> list of game feature tensors
        
        self.predictions = []
        self.actuals = []
        
    def create_game_features(self, team_id, opponent_id, is_home, prev_score_diff=0.0):
        """Create feature vector for a single game"""
        with torch.no_grad():  # Detach from computation graph
            team_emb = self.model.team_embeddings(torch.tensor(team_id, device=self.device))
            opp_emb = self.model.team_embeddings(torch.tensor(opponent_id, device=self.device))
            home_flag = torch.tensor(1.0 if is_home else 0.0, device=self.device)
            score_diff = torch.tensor(prev_score_diff, device=self.device)
            
            # Concatenate all features and detach
            features = torch.cat([team_emb, opp_emb, home_flag.unsqueeze(0), score_diff.unsqueeze(0)])
            return features.clone().detach()  # Ensure no gradients are attached
        
    def predict_game(self, home_team_id, away_team_id):
        self.model.eval()
        with torch.no_grad():
            # Get recent histories (maintaining chronological order)
            home_history = self.team_histories[home_team_id][-self.history_length:]
            away_history = self.team_histories[away_team_id][-self.history_length:]
            
            pred = self.model(home_team_id, away_team_id, home_history, away_history)
            return pred[0, 0].item(), pred[0, 1].item()
            
    def update_with_result(self, home_team_id, away_team_id, actual_home, actual_away):
        self.model.train()
        
        # Get current histories
        home_history = self.team_histories[home_team_id][-self.history_length:]
        away_history = self.team_histories[away_team_id][-self.history_length:]
        
        # Make prediction
        pred = self.model(home_team_id, away_team_id, home_history, away_history)
        target = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)
        
        # Compute loss and update
        loss = self.loss_fn(pred, target)
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()
        
        # Update team histories with this game's result
        # For home team
        home_score_diff = actual_home - actual_away
        home_features = self.create_game_features(home_team_id, away_team_id, True, home_score_diff)
        self.team_histories[home_team_id].append(home_features)
        
        # For away team  
        away_score_diff = actual_away - actual_home
        away_features = self.create_game_features(away_team_id, home_team_id, False, away_score_diff)
        self.team_histories[away_team_id].append(away_features)
        
        # Keep only recent history
        if len(self.team_histories[home_team_id]) > self.history_length * 2:
            self.team_histories[home_team_id] = self.team_histories[home_team_id][-self.history_length:]
        if len(self.team_histories[away_team_id]) > self.history_length * 2:
            self.team_histories[away_team_id] = self.team_histories[away_team_id][-self.history_length:]
            
        return loss.item()

# ==========================================================
# 3. GRU Online training with chronological train/val/test
# ==========================================================
def run_gru_online_train_val_test(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Split chronologically to avoid data leakage
    train_idx = int(n_total * 0.7)
    val_idx = int(n_total * 0.85)

    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # Map team IDs
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    n_teams = len(team2idx)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = GRUOnlineNBAPredictor(n_teams, device, history_length=10)

    # ----------- TRAIN (online, chronological) -----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % 200 == 0:
            recent_preds = predictor.predictions[-200:]
            recent_actuals = predictor.actuals[-200:]
            home_preds = [p[0] for p in recent_preds]
            home_actuals = [a[0] for a in recent_actuals]
            mse = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
            print(f"Train Games {idx+1}: Recent MSE={mse:.2f}, Avg Loss={total_loss/(idx+1):.4f}")

    # ----------- VALIDATION (chronological, no updates to model, but histories continue) -----------
    # Create a copy of histories at end of training
    val_histories = {k: v.copy() for k, v in predictor.team_histories.items()}
    
    val_preds, val_actuals = [], []
    for idx, game in df_val.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append((pred_home, pred_away))
        val_actuals.append((actual_home, actual_away))
        
        # Update histories for next prediction but don't train model
        home_score_diff = actual_home - actual_away
        home_features = predictor.create_game_features(home_id, away_id, True, home_score_diff)
        predictor.team_histories[home_id].append(home_features)
        
        away_score_diff = actual_away - actual_home
        away_features = predictor.create_game_features(away_id, home_id, False, away_score_diff)
        predictor.team_histories[away_id].append(away_features)

    home_preds = [p[0] for p in val_preds]
    home_actuals = [a[0] for a in val_actuals]
    mse_home = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
    away_preds = [p[1] for p in val_preds]
    away_actuals = [a[1] for a in val_actuals]
    mse_away = np.mean([(p-a)**2 for p, a in zip(away_preds, away_actuals)])
    print(f"Validation MSE: home={mse_home:.2f}, away={mse_away:.2f}")

    # ----------- TEST (chronological, final evaluation) -----------
    test_preds, test_actuals = [], []
    for idx, game in df_test.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append((pred_home, pred_away))
        test_actuals.append((actual_home, actual_away))
        
        # Update histories for next prediction
        home_score_diff = actual_home - actual_away
        home_features = predictor.create_game_features(home_id, away_id, True, home_score_diff)
        predictor.team_histories[home_id].append(home_features)
        
        away_score_diff = actual_away - actual_home
        away_features = predictor.create_game_features(away_id, home_id, False, away_score_diff)
        predictor.team_histories[away_id].append(away_features)

    home_preds = [p[0] for p in test_preds]
    home_actuals = [a[0] for a in test_actuals]
    mse_home = np.mean([(p-a)**2 for p, a in zip(home_preds, home_actuals)])
    away_preds = [p[1] for p in test_preds]
    away_actuals = [a[1] for a in test_actuals]
    mse_away = np.mean([(p-a)**2 for p, a in zip(away_preds, away_actuals)])
    print(f"Test MSE: home={mse_home:.2f}, away={mse_away:.2f}")

# Usage:
run_gru_online_train_val_test(df)

Train Games 200: Recent MSE=188.37, Avg Loss=196.1544
Train Games 400: Recent MSE=189.04, Avg Loss=185.1839
Train Games 600: Recent MSE=118.14, Avg Loss=166.1200
Train Games 800: Recent MSE=176.36, Avg Loss=164.6305
Train Games 1000: Recent MSE=164.14, Avg Loss=161.8124
Train Games 1200: Recent MSE=168.28, Avg Loss=161.8153
Train Games 1400: Recent MSE=148.13, Avg Loss=160.7568
Train Games 1600: Recent MSE=152.20, Avg Loss=158.9834
Train Games 1800: Recent MSE=158.33, Avg Loss=158.9554
Train Games 2000: Recent MSE=141.21, Avg Loss=158.5870
Train Games 2200: Recent MSE=186.77, Avg Loss=160.7264
Train Games 2400: Recent MSE=146.01, Avg Loss=158.7197
Train Games 2600: Recent MSE=145.93, Avg Loss=158.0187
Train Games 2800: Recent MSE=149.77, Avg Loss=158.2647
Train Games 3000: Recent MSE=139.77, Avg Loss=158.0977
Train Games 3200: Recent MSE=149.79, Avg Loss=157.8555
Train Games 3400: Recent MSE=153.95, Avg Loss=157.6371
Train Games 3600: Recent MSE=134.81, Avg Loss=156.2531
Train Games 38

In [37]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. True Online NBA Model with LayerNorm
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=8, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))

        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim*2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),  # safe for online
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, home_ids, away_ids):
        home_emb = self.team_embeddings(home_ids)
        away_emb = self.team_embeddings(away_ids)
        batch_size = home_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        # Sigmoid scaling to NBA range [80,150]
        scores = torch.sigmoid(raw_scores) * 70 + 80
        return scores

# ==========================================================
# 2. Online Predictor (no target normalization)
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams, embedding_dim=8).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_id, away_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0,0].item(), pred[0,1].item()

    def update_with_result(self, home_id, away_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_id], device=self.device)
        away_tensor = torch.tensor([away_id], device=self.device)
        target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target_tensor)

        # Adaptive learning rate
        games_seen = len(self.predictions)
        lr = max(0.001, 0.01 * (0.995 ** games_seen))
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()

        return loss.item()

# ==========================================================
# 3. Run Online Train / Val / Test
# ==========================================================
def run_online_train_val_test(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Chronological splits
    train_idx = int(n_total * 0.7)
    val_idx   = int(n_total * 0.85)
    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # Map team IDs
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    n_teams = len(team2idx)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = TrueOnlineNBAPredictor(n_teams, device)

    # ---------- Train ----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        # Predict
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        # Update
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % 200 == 0:
            print(f"Train Game {idx+1}: Avg Loss={total_loss/(idx+1):.4f}")

    # ---------- Validation ----------
    val_preds, val_targets = [], []
    for _, game in df_val.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append([pred_home, pred_away])
        val_targets.append([game['home_pts'], game['away_pts']])
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)
    mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
    mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
    print(f"Validation MSE: home={mse_home:.2f}, away={mse_away:.2f}")

    # ---------- Test ----------
    test_preds, test_targets = [], []
    for _, game in df_test.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append([pred_home, pred_away])
        test_targets.append([game['home_pts'], game['away_pts']])
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
    mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
    print(f"Test MSE: home={mse_home:.2f}, away={mse_away:.2f}")

# Usage:
run_online_train_val_test(df)


Train Game 200: Avg Loss=189.7991
Train Game 400: Avg Loss=180.0519
Train Game 600: Avg Loss=162.5402
Train Game 800: Avg Loss=160.6295
Train Game 1000: Avg Loss=157.7466
Train Game 1200: Avg Loss=158.4203
Train Game 1400: Avg Loss=157.3864
Train Game 1600: Avg Loss=155.9192
Train Game 1800: Avg Loss=155.4704
Train Game 2000: Avg Loss=155.0303
Train Game 2200: Avg Loss=156.6811
Train Game 2400: Avg Loss=154.5474
Train Game 2600: Avg Loss=154.1713
Train Game 2800: Avg Loss=154.4668
Train Game 3000: Avg Loss=154.0116
Train Game 3200: Avg Loss=153.7095
Train Game 3400: Avg Loss=153.4992
Train Game 3600: Avg Loss=152.1388
Train Game 3800: Avg Loss=151.1957
Train Game 4000: Avg Loss=149.8732
Train Game 4200: Avg Loss=149.1194
Train Game 4400: Avg Loss=149.2792
Train Game 4600: Avg Loss=149.5534
Train Game 4800: Avg Loss=149.2129
Validation MSE: home=160.23, away=157.47
Test MSE: home=163.21, away=166.73


In [24]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. True Online NBA Model with LayerNorm
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=8, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim*2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, home_ids, away_ids):
        home_emb = self.team_embeddings(home_ids)
        away_emb = self.team_embeddings(away_ids)
        batch_size = home_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA range [80,150]
        return scores

# ==========================================================
# 2. Online Predictor
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams, embedding_dim=8).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_id, away_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0,0].item(), pred[0,1].item()

    def update_with_result(self, home_id, away_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_id], device=self.device)
        away_tensor = torch.tensor([away_id], device=self.device)
        target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target_tensor)

        # Adaptive learning rate
        games_seen = len(self.predictions)
        lr = max(0.001, 0.01 * (0.995 ** games_seen))
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()

        return loss.item()

# ==========================================================
# 3. Run Online Train / Val / Test with MSE + R^2
# ==========================================================
def run_online_train_val_test(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Chronological splits
    train_idx = int(n_total * 0.7)
    val_idx   = int(n_total * 0.85)
    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # Map team IDs
    all_team_ids = pd.concat([df['home_team_id'], df['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(all_team_ids))}
    n_teams = len(team2idx)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = TrueOnlineNBAPredictor(n_teams, device)

    # ---------- Train ----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        # Predict
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        # Update
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % 200 == 0:
            print(f"Train Game {idx+1}: Avg Loss={total_loss/(idx+1):.4f}")

    # ---------- Validation ----------
    val_preds, val_targets = [], []
    for _, game in df_val.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append([pred_home, pred_away])
        val_targets.append([game['home_pts'], game['away_pts']])
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)
    mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
    mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
    r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2) / np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
    r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2) / np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)
    print(f"Validation: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

    # ---------- Test ----------
    test_preds, test_targets = [], []
    for _, game in df_test.iterrows():
        home_id = team2idx[game['home_team_id']]
        away_id = team2idx[game['away_team_id']]
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append([pred_home, pred_away])
        test_targets.append([game['home_pts'], game['away_pts']])
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
    mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
    r2_home = 1 - np.sum((test_targets[:,0]-test_preds[:,0])**2) / np.sum((test_targets[:,0]-np.mean(test_targets[:,0]))**2)
    r2_away = 1 - np.sum((test_targets[:,1]-test_preds[:,1])**2) / np.sum((test_targets[:,1]-np.mean(test_targets[:,1]))**2)
    print(f"Test: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

# Usage:
run_online_train_val_test(df)


Train Game 200: Avg Loss=187.3120
Train Game 400: Avg Loss=178.4296
Train Game 600: Avg Loss=161.3433
Train Game 800: Avg Loss=160.9565
Train Game 1000: Avg Loss=158.6793
Train Game 1200: Avg Loss=159.1755
Train Game 1400: Avg Loss=158.4467
Train Game 1600: Avg Loss=156.5986
Train Game 1800: Avg Loss=156.8461
Train Game 2000: Avg Loss=156.2800
Train Game 2200: Avg Loss=158.0838
Train Game 2400: Avg Loss=156.1027
Train Game 2600: Avg Loss=155.5782
Train Game 2800: Avg Loss=155.7753
Train Game 3000: Avg Loss=155.2221
Train Game 3200: Avg Loss=154.4720
Train Game 3400: Avg Loss=154.2460
Train Game 3600: Avg Loss=153.0478
Train Game 3800: Avg Loss=152.0779
Train Game 4000: Avg Loss=150.6279
Train Game 4200: Avg Loss=149.7254
Train Game 4400: Avg Loss=149.9458
Train Game 4600: Avg Loss=150.3463
Train Game 4800: Avg Loss=149.8499
Validation: MSE home=161.62, away=162.23, R² home=0.006, away=0.010
Test: MSE home=168.15, away=165.30, R² home=-0.033, away=-0.002


In [29]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. True Online NBA Model with LayerNorm
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=8, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim*2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, home_ids, away_ids):
        home_emb = self.team_embeddings(home_ids)
        away_emb = self.team_embeddings(away_ids)
        batch_size = home_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA range [80,150]
        return scores

# ==========================================================
# 2. Online Predictor
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams, embedding_dim=8).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_id, away_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0,0].item(), pred[0,1].item()

    def update_with_result(self, home_id, away_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_id], device=self.device)
        away_tensor = torch.tensor([away_id], device=self.device)
        target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target_tensor)

        # Adaptive learning rate
        games_seen = len(self.predictions)
        lr = 0.01 / np.sqrt(1 + games_seen / 500)
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()

        return loss.item()

# ==========================================================
# 3. Data Leakage Free Rolling Train/Val/Test
# ==========================================================
def run_online_train_val_test_rolling(df, window=200):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Chronological splits
    train_idx = int(n_total * 0.7)
    val_idx   = int(n_total * 0.85)
    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # FIXED: Map team IDs using ONLY training data to prevent data leakage
    train_team_ids = pd.concat([df_train['home_team_id'], df_train['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(train_team_ids))}
    n_teams = len(team2idx)
    
    print(f"Training teams: {n_teams}")
    print(f"Training games: {len(df_train)}, Validation: {len(df_val)}, Test: {len(df_test)}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = TrueOnlineNBAPredictor(n_teams, device)

    # Helper function to handle unseen teams
    def get_team_idx(team_id, team2idx, default_idx=0):
        """Get team index, use default (index 0) for unseen teams"""
        return team2idx.get(team_id, default_idx)

    # Track unseen teams for reporting
    unseen_teams_val = set()
    unseen_teams_test = set()

    # ---------- Train with rolling metrics ----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = team2idx[game['home_team_id']]  # All training teams are guaranteed to be in mapping
        away_id = team2idx[game['away_team_id']]
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        # Predict & store
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        # Update model
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        # Rolling metrics every `window` games
        if (idx + 1) % window == 0:
            recent_preds = np.array(predictor.predictions[-window:])
            recent_actuals = np.array(predictor.actuals[-window:])
            
            mse_home = np.mean((recent_preds[:,0]-recent_actuals[:,0])**2)
            mse_away = np.mean((recent_preds[:,1]-recent_actuals[:,1])**2)
            
            r2_home = 1 - np.sum((recent_actuals[:,0]-recent_preds[:,0])**2) / np.sum((recent_actuals[:,0]-np.mean(recent_actuals[:,0]))**2)
            r2_away = 1 - np.sum((recent_actuals[:,1]-recent_preds[:,1])**2) / np.sum((recent_actuals[:,1]-np.mean(recent_actuals[:,1]))**2)

            print(f"Games {idx+1}: Rolling MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

    # ---------- Validation ----------
    val_preds, val_targets = [], []
    for _, game in df_val.iterrows():
        home_id = get_team_idx(game['home_team_id'], team2idx, default_idx=0)
        away_id = get_team_idx(game['away_team_id'], team2idx, default_idx=0)
        
        # Track unseen teams
        if game['home_team_id'] not in team2idx:
            unseen_teams_val.add(game['home_team_id'])
        if game['away_team_id'] not in team2idx:
            unseen_teams_val.add(game['away_team_id'])
            
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append([pred_home, pred_away])
        val_targets.append([game['home_pts'], game['away_pts']])
        
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)
    mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
    mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
    r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2) / np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
    r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2) / np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)
    
    print(f"\nValidation: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")
    if unseen_teams_val:
        print(f"Validation unseen teams ({len(unseen_teams_val)}): {sorted(list(unseen_teams_val))[:5]}{'...' if len(unseen_teams_val) > 5 else ''}")

    # ---------- Test ----------
    test_preds, test_targets = [], []
    for _, game in df_test.iterrows():
        home_id = get_team_idx(game['home_team_id'], team2idx, default_idx=0)
        away_id = get_team_idx(game['away_team_id'], team2idx, default_idx=0)
        
        # Track unseen teams
        if game['home_team_id'] not in team2idx:
            unseen_teams_test.add(game['home_team_id'])
        if game['away_team_id'] not in team2idx:
            unseen_teams_test.add(game['away_team_id'])
            
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append([pred_home, pred_away])
        test_targets.append([game['home_pts'], game['away_pts']])
        
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
    mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
    r2_home = 1 - np.sum((test_targets[:,0]-test_preds[:,0])**2) / np.sum((test_targets[:,0]-np.mean(test_targets[:,0]))**2)
    r2_away = 1 - np.sum((test_targets[:,1]-test_preds[:,1])**2) / np.sum((test_targets[:,1]-np.mean(test_targets[:,1]))**2)
    
    print(f"\nTest: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")
    if unseen_teams_test:
        print(f"Test unseen teams ({len(unseen_teams_test)}): {sorted(list(unseen_teams_test))[:5]}{'...' if len(unseen_teams_test) > 5 else ''}")
    

# Usage:
run_online_train_val_test_rolling(df, window=200)

Training teams: 30
Training games: 4937, Validation: 1058, Test: 1059
Games 200: Rolling MSE home=191.80, away=194.37, R² home=-0.219, away=-0.186
Games 400: Rolling MSE home=183.80, away=160.09, R² home=-0.046, away=-0.046
Games 600: Rolling MSE home=117.19, away=138.37, R² home=-0.016, away=-0.035
Games 800: Rolling MSE home=178.04, away=144.40, R² home=-0.025, away=-0.015
Games 1000: Rolling MSE home=162.97, away=137.02, R² home=-0.011, away=-0.010
Games 1200: Rolling MSE home=168.18, away=155.22, R² home=-0.005, away=-0.008
Games 1400: Rolling MSE home=147.28, away=159.80, R² home=-0.007, away=-0.010
Games 1600: Rolling MSE home=149.09, away=137.92, R² home=-0.009, away=-0.005
Games 1800: Rolling MSE home=161.41, away=159.80, R² home=-0.039, away=-0.028
Games 2000: Rolling MSE home=139.64, away=166.59, R² home=-0.006, away=-0.016
Games 2200: Rolling MSE home=184.74, away=176.92, R² home=-0.012, away=-0.002
Games 2400: Rolling MSE home=149.43, away=130.79, R² home=-0.090, away=-0.14

In [ ]:
    import torch
    import torch.nn as nn
    import numpy as np
    import pandas as pd

    # ==========================================================
    # 1. True Online NBA Model with LayerNorm
    # ==========================================================
    class TrueOnlineNBAModel(nn.Module):
        def __init__(self, n_teams, embedding_dim=8, hidden_dim=64):
            super().__init__()
            self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
            self.home_advantage = nn.Parameter(torch.tensor(2.5))  # initial prior

            nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

            self.net = nn.Sequential(
                nn.Linear(embedding_dim*2 + 1, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 2)
            )

        def forward(self, home_ids, away_ids):
            home_emb = self.team_embeddings(home_ids)
            away_emb = self.team_embeddings(away_ids)
            batch_size = home_ids.size(0)
            home_adv = self.home_advantage.expand(batch_size, 1)
            x = torch.cat([home_emb, away_emb, home_adv], dim=1)
            out = self.net(x)
            # No global target scaling
            return out  # raw predicted points

    # ==========================================================
    # 2. Online predictor
    # ==========================================================
    class TrueOnlineNBAPredictor:
        def __init__(self, n_teams, device='cpu'):
            self.device = device
            self.model = TrueOnlineNBAModel(n_teams, embedding_dim=8).to(device)
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-4)
            self.loss_fn = nn.MSELoss()
            self.predictions = []
            self.actuals = []

        def predict_game(self, home_id, away_id):
            self.model.eval()
            with torch.no_grad():
                home_tensor = torch.tensor([home_id], device=self.device)
                away_tensor = torch.tensor([away_id], device=self.device)
                pred = self.model(home_tensor, away_tensor)
                return pred[0,0].item(), pred[0,1].item()

        def update_with_result(self, home_id, away_id, actual_home, actual_away):
            self.model.train()
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

            pred = self.model(home_tensor, away_tensor)
            loss = self.loss_fn(pred, target_tensor)

            # Optional: adaptive learning rate
            games_seen = len(self.predictions)
            lr = 0.01 / np.sqrt(1 + games_seen / 1000)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr

            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            return loss.item()

    # ==========================================================
    # 3. Rolling train/val/test
    # ==========================================================
    def run_online_train_val_test(df, window=200):
        df_sorted = df.sort_values('date').reset_index(drop=True)
        n_total = len(df_sorted)

        # Chronological splits
        train_idx = int(n_total * 0.7)
        val_idx   = int(n_total * 0.85)
        df_train = df_sorted.iloc[:train_idx]
        df_val   = df_sorted.iloc[train_idx:val_idx]
        df_test  = df_sorted.iloc[val_idx:]

        # Map team IDs using training data only
        train_team_ids = pd.concat([df_train['home_team_id'], df_train['away_team_id']]).unique()
        team2idx = {tid:i for i, tid in enumerate(sorted(train_team_ids))}
        n_teams = len(team2idx)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        predictor = TrueOnlineNBAPredictor(n_teams, device)

        def get_team_idx(team_id, team2idx, default_idx=0):
            return team2idx.get(team_id, default_idx)

        # ---------- Train ----------
        total_loss = 0
        for idx, game in df_train.iterrows():
            home_id = team2idx[game['home_team_id']]
            away_id = team2idx[game['away_team_id']]
            actual_home = game['home_pts']
            actual_away = game['away_pts']

            # Predict and update
            pred_home, pred_away = predictor.predict_game(home_id, away_id)
            predictor.predictions.append((pred_home, pred_away))
            predictor.actuals.append((actual_home, actual_away))
            loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
            total_loss += loss

            # Rolling metrics
            if (idx+1) % window == 0:
                recent_preds = np.array(predictor.predictions[-window:])
                recent_actuals = np.array(predictor.actuals[-window:])
                mse_home = np.mean((recent_preds[:,0]-recent_actuals[:,0])**2)
                mse_away = np.mean((recent_preds[:,1]-recent_actuals[:,1])**2)
                r2_home = 1 - np.sum((recent_actuals[:,0]-recent_preds[:,0])**2)/np.sum((recent_actuals[:,0]-np.mean(recent_actuals[:,0]))**2)
                r2_away = 1 - np.sum((recent_actuals[:,1]-recent_preds[:,1])**2)/np.sum((recent_actuals[:,1]-np.mean(recent_actuals[:,1]))**2)
                print(f"Games {idx+1}: Rolling MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

        # ---------- Validation ----------
        val_preds, val_targets = [], []
        for _, game in df_val.iterrows():
            home_id = get_team_idx(game['home_team_id'], team2idx)
            away_id = get_team_idx(game['away_team_id'], team2idx)
            pred_home, pred_away = predictor.predict_game(home_id, away_id)
            val_preds.append([pred_home, pred_away])
            val_targets.append([game['home_pts'], game['away_pts']])
        val_preds = np.array(val_preds)
        val_targets = np.array(val_targets)
        mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
        mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
        r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2)/np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
        r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2)/np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)
        print(f"\nValidation: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

        # ---------- Test ----------
        test_preds, test_targets = [], []
        for _, game in df_test.iterrows():
            home_id = get_team_idx(game['home_team_id'], team2idx)
            away_id = get_team_idx(game['away_team_id'], team2idx)
            pred_home, pred_away = predictor.predict_game(home_id, away_id)
            test_preds.append([pred_home, pred_away])
            test_targets.append([game['home_pts'], game['away_pts']])
        test_preds = np.array(test_preds)
        test_targets = np.array(test_targets)
        mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
        mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
        r2_home = 1 - np.sum((test_targets[:,0]-test_preds[:,0])**2)/np.sum((test_targets[:,0]-np.mean(test_targets[:,0]))**2)
        r2_away = 1 - np.sum((test_targets[:,1]-test_preds[:,1])**2)/np.sum((test_targets[:,1]-np.mean(test_targets[:,1]))**2)
        print(f"\nTest: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}")

    # Usage:
    run_online_train_val_test(df, window=200)


Games 200: Rolling MSE home=3628.99, away=3424.05, R² home=-22.065, away=-19.896
Games 400: Rolling MSE home=187.64, away=158.02, R² home=-0.068, away=-0.033
Games 600: Rolling MSE home=126.48, away=145.52, R² home=-0.097, away=-0.088
Games 800: Rolling MSE home=177.16, away=149.37, R² home=-0.020, away=-0.050
Games 1000: Rolling MSE home=167.16, away=134.44, R² home=-0.037, away=0.009
Games 1200: Rolling MSE home=173.94, away=159.81, R² home=-0.039, away=-0.038
Games 1400: Rolling MSE home=152.86, away=169.54, R² home=-0.046, away=-0.072
Games 1600: Rolling MSE home=159.45, away=140.72, R² home=-0.079, away=-0.026
Games 1800: Rolling MSE home=160.25, away=157.89, R² home=-0.031, away=-0.015
Games 2000: Rolling MSE home=143.68, away=165.88, R² home=-0.035, away=-0.012
Games 2200: Rolling MSE home=176.23, away=171.36, R² home=0.034, away=0.029
Games 2400: Rolling MSE home=146.88, away=123.45, R² home=-0.071, away=-0.085
Games 2600: Rolling MSE home=149.96, away=154.87, R² home=-0.036, a

In [36]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. True Online NBA Model with LayerNorm
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=12, hidden_dim=128):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))
        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.1)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim*2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 2)  # Predict home and away scores
        )

    def forward(self, home_ids, away_ids):
        home_emb = self.team_embeddings(home_ids)
        away_emb = self.team_embeddings(away_ids)
        batch_size = home_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        return self.net(x)

# ==========================================================
# 2. Online Predictor
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams, embedding_dim=12).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.005, weight_decay=1e-3)
        self.loss_fn = nn.HuberLoss(delta=10.0)  # Huber loss for robustness
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=100, factor=0.5)
        self.predictions = []
        self.actuals = []
        self.batch_losses = []

    def predict_game(self, home_id, away_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0,0].item(), pred[0,1].item()

    def update_with_result(self, home_id, away_id, actual_home, actual_away, batch_size=10):
        self.model.train()
        home_tensor = torch.tensor([home_id], device=self.device)
        away_tensor = torch.tensor([away_id], device=self.device)
        target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target_tensor)
        self.batch_losses.append(loss)

        loss.backward()
        if (len(self.predictions) + 1) % batch_size == 0:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)
            self.optimizer.step()
            self.optimizer.zero_grad()
            avg_loss = sum([l.item() for l in self.batch_losses]) / len(self.batch_losses)
            self.scheduler.step(avg_loss)
            self.batch_losses = []
        return loss.item()

# ==========================================================
# 3. Rolling Train/Val/Test
# ==========================================================
def run_online_train_val_test(df, window=200):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)
    train_idx = int(n_total * 0.7)  # ~4,937 games
    val_idx = int(n_total * 0.85)   # ~5,995 games
    df_train = df_sorted.iloc[:train_idx]
    df_val = df_sorted.iloc[train_idx:val_idx]
    df_test = df_sorted.iloc[val_idx:]

    # Map team IDs using training data
    train_team_ids = pd.concat([df_train['home_team_id'], df_train['away_team_id']]).unique()
    team2idx = {tid: i for i, tid in enumerate(sorted(train_team_ids))}
    n_teams = len(team2idx)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    predictor = TrueOnlineNBAPredictor(n_teams, device)

    def get_team_idx(team_id, team2idx):
        return team2idx.get(team_id, None)  # Return None for unseen teams

    # ---------- Train ----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = get_team_idx(game['home_team_id'], team2idx)
        away_id = get_team_idx(game['away_team_id'], team2idx)
        if home_id is None or away_id is None:
            continue  # Skip games with unseen teams
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % window == 0:
            recent_preds = np.array(predictor.predictions[-window:])
            recent_actuals = np.array(predictor.actuals[-window:])
            mse_home = np.mean((recent_preds[:,0]-recent_actuals[:,0])**2)
            mse_away = np.mean((recent_preds[:,1]-recent_actuals[:,1])**2)
            r2_home = 1 - np.sum((recent_actuals[:,0]-recent_preds[:,0])**2)/np.sum((recent_actuals[:,0]-np.mean(recent_actuals[:,0]))**2)
            r2_away = 1 - np.sum((recent_actuals[:,1]-recent_preds[:,1])**2)/np.sum((recent_actuals[:,1]-np.mean(recent_actuals[:,1]))**2)
            mae_home = np.mean(np.abs(recent_preds[:,0]-recent_actuals[:,0]))
            mae_away = np.mean(np.abs(recent_preds[:,1]-recent_actuals[:,1]))
            winner_acc = np.mean((recent_preds[:,0] > recent_preds[:,1]) == (recent_actuals[:,0] > recent_actuals[:,1]))
            print(f"Games {idx+1}: Rolling MSE home={mse_home:.2f}, away={mse_away:.2f}, "
                  f"R² home={r2_home:.3f}, away={r2_away:.3f}, MAE home={mae_home:.2f}, away={mae_away:.2f}, "
                  f"Winner Acc={winner_acc:.3f}")

            # Validation check
            val_preds, val_targets = [], []
            for _, game in df_val.iterrows():
                home_id = get_team_idx(game['home_team_id'], team2idx)
                away_id = get_team_idx(game['away_team_id'], team2idx)
                if home_id is None or away_id is None:
                    continue
                pred_home, pred_away = predictor.predict_game(home_id, away_id)
                val_preds.append([pred_home, pred_away])
                val_targets.append([game['home_pts'], game['away_pts']])
            if val_preds:
                val_preds = np.array(val_preds)
                val_targets = np.array(val_targets)
                mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
                mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
                r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2)/np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
                r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2)/np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)
                mae_home = np.mean(np.abs(val_preds[:,0]-val_targets[:,0]))
                mae_away = np.mean(np.abs(val_preds[:,1]-val_targets[:,1]))
                winner_acc = np.mean((val_preds[:,0] > val_preds[:,1]) == (val_targets[:,0] > val_targets[:,1]))
                print(f"Games {idx+1}: Validation MSE home={mse_home:.2f}, away={mse_away:.2f}, "
                      f"R² home={r2_home:.3f}, away={r2_away:.3f}, MAE home={mae_home:.2f}, away={mae_away:.2f}, "
                      f"Winner Acc={winner_acc:.3f}")

    # ---------- Validation ----------
    val_preds, val_targets = [], []
    for _, game in df_val.iterrows():
        home_id = get_team_idx(game['home_team_id'], team2idx)
        away_id = get_team_idx(game['away_team_id'], team2idx)
        if home_id is None or away_id is None:
            continue
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append([pred_home, pred_away])
        val_targets.append([game['home_pts'], game['away_pts']])
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)
    mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
    mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
    r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2)/np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
    r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2)/np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)
    mae_home = np.mean(np.abs(val_preds[:,0]-val_targets[:,0]))
    mae_away = np.mean(np.abs(val_preds[:,1]-val_targets[:,1]))
    winner_acc = np.mean((val_preds[:,0] > val_preds[:,1]) == (val_targets[:,0] > val_targets[:,1]))
    print(f"\nValidation: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}, "
          f"MAE home={mae_home:.2f}, away={mae_away:.2f}, Winner Acc={winner_acc:.3f}")

    # ---------- Test ----------
    test_preds, test_targets = [], []
    for _, game in df_test.iterrows():
        home_id = get_team_idx(game['home_team_id'], team2idx)
        away_id = get_team_idx(game['away_team_id'], team2idx)
        if home_id is None or away_id is None:
            continue
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append([pred_home, pred_away])
        test_targets.append([game['home_pts'], game['away_pts']])
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
    mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
    r2_home = 1 - np.sum((test_targets[:,0]-test_preds[:,0])**2)/np.sum((test_targets[:,0]-np.mean(test_targets[:,0]))**2)
    r2_away = 1 - np.sum((test_targets[:,1]-test_preds[:,1])**2)/np.sum((test_targets[:,1]-np.mean(test_targets[:,1]))**2)
    mae_home = np.mean(np.abs(test_preds[:,0]-test_targets[:,0]))
    mae_away = np.mean(np.abs(test_preds[:,1]-test_targets[:,1]))
    winner_acc = np.mean((test_preds[:,0] > test_preds[:,1]) == (test_targets[:,0] > test_targets[:,1]))
    print(f"\nTest: MSE home={mse_home:.2f}, away={mse_away:.2f}, R² home={r2_home:.3f}, away={r2_away:.3f}, "
          f"MAE home={mae_home:.2f}, away={mae_away:.2f}, Winner Acc={winner_acc:.3f}")

# Usage:
run_online_train_val_test(df, window=200)

Games 200: Rolling MSE home=11289.07, away=11081.88, R² home=-70.751, away=-66.631, MAE home=105.48, away=104.46, Winner Acc=0.585
Games 200: Validation MSE home=11153.23, away=10932.45, R² home=-67.861, away=-65.611, MAE home=104.84, away=103.77, Winner Acc=0.533
Games 400: Rolling MSE home=9695.21, away=9143.65, R² home=-54.158, away=-58.757, MAE home=97.48, away=94.80, Winner Acc=0.545
Games 400: Validation MSE home=9078.01, away=8891.83, R² home=-55.048, away=-53.178, MAE home=94.42, away=93.42, Winner Acc=0.533
Games 600: Rolling MSE home=7398.51, away=7119.22, R² home=-62.510, away=-51.991, MAE home=85.27, away=83.48, Winner Acc=0.570
Games 600: Validation MSE home=6748.14, away=6578.82, R² home=-40.663, away=-39.084, MAE home=81.16, away=80.09, Winner Acc=0.533
Games 800: Rolling MSE home=5502.53, away=5324.43, R² home=-30.940, away=-36.553, MAE home=72.91, away=71.81, Winner Acc=0.520
Games 800: Validation MSE home=4390.34, away=4264.45, R² home=-26.106, away=-24.983, MAE home=

In [40]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# ==========================================================
# 1. True Online NBA Model with LayerNorm
# ==========================================================
class TrueOnlineNBAModel(nn.Module):
    def __init__(self, n_teams, embedding_dim=8, hidden_dim=64):
        super().__init__()
        self.team_embeddings = nn.Embedding(n_teams, embedding_dim)
        self.home_advantage = nn.Parameter(torch.tensor(2.5))

        nn.init.normal_(self.team_embeddings.weight, mean=0.0, std=0.01)

        self.net = nn.Sequential(
            nn.Linear(embedding_dim*2 + 1, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, home_ids, away_ids):
        home_emb = self.team_embeddings(home_ids)
        away_emb = self.team_embeddings(away_ids)
        batch_size = home_ids.size(0)
        home_adv = self.home_advantage.expand(batch_size, 1)
        x = torch.cat([home_emb, away_emb, home_adv], dim=1)
        raw_scores = self.net(x)
        scores = torch.sigmoid(raw_scores) * 70 + 80  # Map to NBA range [80,150]
        return scores

# ==========================================================
# 2. Online Predictor (no target normalization)
# ==========================================================
class TrueOnlineNBAPredictor:
    def __init__(self, n_teams, device='cpu'):
        self.device = device
        self.model = TrueOnlineNBAModel(n_teams, embedding_dim=8).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-3)
        self.loss_fn = nn.MSELoss()
        self.predictions = []
        self.actuals = []

    def predict_game(self, home_id, away_id):
        self.model.eval()
        with torch.no_grad():
            home_tensor = torch.tensor([home_id], device=self.device)
            away_tensor = torch.tensor([away_id], device=self.device)
            pred = self.model(home_tensor, away_tensor)
            return pred[0,0].item(), pred[0,1].item()

    def update_with_result(self, home_id, away_id, actual_home, actual_away):
        self.model.train()
        home_tensor = torch.tensor([home_id], device=self.device)
        away_tensor = torch.tensor([away_id], device=self.device)
        target_tensor = torch.tensor([[actual_home, actual_away]], dtype=torch.float32, device=self.device)

        pred = self.model(home_tensor, away_tensor)
        loss = self.loss_fn(pred, target_tensor)

        # Adaptive learning rate
        games_seen = len(self.predictions)
        lr = max(0.001, 0.01 * (0.995 ** games_seen))
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()
        return loss.item()

# ==========================================================
# 3. Run Online Train / Val / Test with leakage-free embeddings
# ==========================================================
def run_online_train_val_test_safe(df):
    df_sorted = df.sort_values('date').reset_index(drop=True)
    n_total = len(df_sorted)

    # Chronological splits
    train_idx = int(n_total * 0.7)
    val_idx   = int(n_total * 0.85)
    df_train = df_sorted.iloc[:train_idx]
    df_val   = df_sorted.iloc[train_idx:val_idx]
    df_test  = df_sorted.iloc[val_idx:]

    # Map only training teams
    train_team_ids = pd.concat([df_train['home_team_id'], df_train['away_team_id']]).unique()
    team2idx = {tid: i+1 for i, tid in enumerate(sorted(train_team_ids))}  # +1 to reserve 0 for unseen
    n_teams = len(team2idx) + 1  # Include default embedding at index 0

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predictor = TrueOnlineNBAPredictor(n_teams, device)

    # Helper to map unseen teams → 0
    def get_team_idx(team_id):
        return team2idx.get(team_id, 0)

    # ---------- Train ----------
    total_loss = 0
    for idx, game in df_train.iterrows():
        home_id = get_team_idx(game['home_team_id'])
        away_id = get_team_idx(game['away_team_id'])
        actual_home = game['home_pts']
        actual_away = game['away_pts']

        # Predict
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        predictor.predictions.append((pred_home, pred_away))
        predictor.actuals.append((actual_home, actual_away))

        # Update
        loss = predictor.update_with_result(home_id, away_id, actual_home, actual_away)
        total_loss += loss

        if (idx+1) % 200 == 0:
            print(f"Train Game {idx+1}: Avg Loss={total_loss/(idx+1):.4f}")

    # ---------- Validation ----------
    val_preds, val_targets = [], []
    for _, game in df_val.iterrows():
        home_id = get_team_idx(game['home_team_id'])
        away_id = get_team_idx(game['away_team_id'])
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        val_preds.append([pred_home, pred_away])
        val_targets.append([game['home_pts'], game['away_pts']])
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)

    mse_home = np.mean((val_preds[:,0]-val_targets[:,0])**2)
    mse_away = np.mean((val_preds[:,1]-val_targets[:,1])**2)
    r2_home = 1 - np.sum((val_targets[:,0]-val_preds[:,0])**2)/np.sum((val_targets[:,0]-np.mean(val_targets[:,0]))**2)
    r2_away = 1 - np.sum((val_targets[:,1]-val_preds[:,1])**2)/np.sum((val_targets[:,1]-np.mean(val_targets[:,1]))**2)

    print(f"\nValidation MSE: home={mse_home:.2f}, away={mse_away:.2f}")
    print(f"Validation R²: home={r2_home:.3f}, away={r2_away:.3f}")

    # ---------- Test ----------
    test_preds, test_targets = [], []
    for _, game in df_test.iterrows():
        home_id = get_team_idx(game['home_team_id'])
        away_id = get_team_idx(game['away_team_id'])
        pred_home, pred_away = predictor.predict_game(home_id, away_id)
        test_preds.append([pred_home, pred_away])
        test_targets.append([game['home_pts'], game['away_pts']])
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)

    mse_home = np.mean((test_preds[:,0]-test_targets[:,0])**2)
    mse_away = np.mean((test_preds[:,1]-test_targets[:,1])**2)
    r2_home = 1 - np.sum((test_targets[:,0]-test_preds[:,0])**2)/np.sum((test_targets[:,0]-np.mean(test_targets[:,0]))**2)
    r2_away = 1 - np.sum((test_targets[:,1]-test_preds[:,1])**2)/np.sum((test_targets[:,1]-np.mean(test_targets[:,1]))**2)

    print(f"\nTest MSE: home={mse_home:.2f}, away={mse_away:.2f}")
    print(f"Test R²: home={r2_home:.3f}, away={r2_away:.3f}")
    
run_online_train_val_test_safe(df)



Train Game 200: Avg Loss=188.3954
Train Game 400: Avg Loss=179.0251
Train Game 600: Avg Loss=161.8174
Train Game 800: Avg Loss=161.0585
Train Game 1000: Avg Loss=158.8115
Train Game 1200: Avg Loss=159.2936
Train Game 1400: Avg Loss=158.5678
Train Game 1600: Avg Loss=156.9958
Train Game 1800: Avg Loss=156.9715
Train Game 2000: Avg Loss=156.6875
Train Game 2200: Avg Loss=158.4799
Train Game 2400: Avg Loss=156.4665
Train Game 2600: Avg Loss=155.9740
Train Game 2800: Avg Loss=156.1363
Train Game 3000: Avg Loss=155.4894
Train Game 3200: Avg Loss=154.8061
Train Game 3400: Avg Loss=154.6763
Train Game 3600: Avg Loss=153.2893
Train Game 3800: Avg Loss=152.2395
Train Game 4000: Avg Loss=150.7190
Train Game 4200: Avg Loss=149.7286
Train Game 4400: Avg Loss=149.8612
Train Game 4600: Avg Loss=150.3034
Train Game 4800: Avg Loss=149.7770

Validation MSE: home=160.65, away=162.26
Validation R²: home=0.008, away=0.011

Test MSE: home=168.59, away=164.95
Test R²: home=-0.032, away=-0.002
